# Development Notebook

### Imports

In [1]:
# -- public imports

from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch

In [2]:
# -- private import
from argminer.data import KaggleDataset

Import of ArgMiner successful


In [3]:
# -- dev imports

%load_ext autoreload
%autoreload 2
from python_files.data import create_labels_doc_level
from python_files.run import train_longformer

Google Drive import failed... continuing anyways...
CUDA device NOT detected. Using CPU...


## Model Configuration

### Tokenizer

In [4]:
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-large')
tokenizer = AutoTokenizer.from_pretrained('google/bigbird-roberta-large', add_prefix_space=True)
# def need an encoding function to categoize the inputs or at least to release them, maybe need to store them together tbh?

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


### Model and Optmizer

In [21]:
config_model = AutoConfig.from_pretrained('google/bigbird-roberta-large') 
config_model.num_labels = 3 # might have to change if we're using the numbered discourse type as well
#config_model.save_pretrained('model')

model = AutoModelForTokenClassification.from_pretrained('google/bigbird-roberta-large', config=config_model)
optimizer = torch.optim.Adam(params=model.parameters())

Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing BigBirdForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were n

## Data Preparation

In [22]:
texts = [
            'Hi my name is Yousef Nami',
            'Hi I am Shirin'
        ]
labels = [
    ['O', 'O', 'O', 'O', 'B-PERS', 'I-PERS'],
    ['O', 'O', 'O', 'B-PERS']
]

# TODO see if bert can accept text inputs?
labels_numeric = [
    [0, 0, 0, 0, 1, 2],
    [0, 0, 0, 1]
]

In [23]:
df = pd.DataFrame({
        'text': texts,
        'labels': labels_numeric
    })

In [80]:
dataset = KaggleDataset(
    df,
    tokenizer,
    max_length=20
)

In [81]:
for (inputs, targets) in dataset:
    print(targets)
    

tensor([0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
"You sef Na mi" -> "BNAME, BNAME, INAME, INAME"

## Data Training

In [82]:
train_loader = DataLoader(dataset, shuffle=True, batch_size=2)

In [86]:
train_longformer(model=model, optimizer=optimizer , epochs=1, train_loader=train_loader)

CUDA device NOT detected. Using CPU...
EPOCH 1 STARTED
---------------
tensor(1.1120, grad_fn=<NllLossBackward0>)
tensor([[[-0.0224, -0.0946,  0.0991],
         [-0.0681, -0.0895,  0.0942],
         [ 0.0401, -0.0928, -0.0653],
         [-0.0004,  0.0033,  0.2081],
         [-0.1006, -0.1141,  0.1016],
         [-0.0174, -0.0251,  0.1580],
         [ 0.0569,  0.0183,  0.0796],
         [ 0.0120, -0.0319,  0.0621],
         [-0.0059, -0.1213,  0.1128],
         [-0.1177, -0.0320,  0.0397],
         [-0.1405, -0.0022,  0.0908],
         [ 0.0113, -0.0479,  0.1648],
         [ 0.0282, -0.1020,  0.0731],
         [-0.0592, -0.0598, -0.0445],
         [ 0.2501, -0.1613, -0.2957],
         [-0.0495, -0.1887,  0.0911],
         [ 0.0622, -0.1628,  0.0522],
         [-0.0403,  0.0131,  0.1195],
         [ 0.0737, -0.0593,  0.0306],
         [-0.0229,  0.0390,  0.1207]],

        [[-0.0111,  0.0077, -0.1385],
         [ 0.1553, -0.1344, -0.0256],
         [ 0.1078,  0.0514,  0.1017],
         [

Exception: 

## Test on Kaggle dataset

In [4]:
PATH = '../../data/kaggle/feedback-prize-2021/train'
FILE_PATH = '../../data/kaggle/feedback-prize-2021/train.csv'

df_texts = create_labels_doc_level(PATH, FILE_PATH)
#df_texts[['text', 'labels']].to_csv('../../../data/kaggle/df_cleaned.csv')

#df_kaggle = pd.read_csv('../../data/kaggle/df_cleaned.csv')
#df_kaggle.head()
df_texts.head()

KeyboardInterrupt: 

In [11]:
df_kaggle = df_texts[['text', 'labels']]
df_kaggle.head()

,text,labels
0,I do agree that some students would benefit fr...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,Should students design a summer project for sc...,"[0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, ..."
2,"Dear State Senator\n\n,\n\nIn the ruels of vot...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 7, 8, ..."
3,People sometimes have a different opinion than...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,"Dear senator,\n\nAs you know the Electoral Col...","[0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [28]:
kaggle_dataset = KaggleDataset(df, tokenizer, max_length=20)

In [29]:
for (inputs, targets) in kaggle_dataset:
    print(inputs)
    break

{'input_ids': tensor([   65, 16003,   717,  1539,   419,   676,   617,   992,   500,  6378,
           66,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


In [30]:
k_train_loader = DataLoader(kaggle_dataset, shuffle=True, batch_size=1)

In [31]:
train_longformer(model=model, optimizer=optimizer, epochs=1, train_loader=k_train_loader)

Attention type 'block_sparse' is not possible if sequence_length: 20 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


EPOCH 1 STARTED
---------------
tensor(1.0604, grad_fn=<NllLossBackward0>)
tensor([[[-0.1543, -0.0606, -0.1969],
         [-0.0912,  0.0348, -0.0372],
         [ 0.0957,  0.2102,  0.1286],
         [ 0.4059,  0.4303,  0.0561],
         [ 0.8208,  0.8731, -0.2832],
         [ 0.0943,  0.1394, -0.4872],
         [ 0.2177,  0.0998, -0.0414],
         [ 0.2388,  0.5826, -0.4362],
         [ 0.2919,  0.1661,  0.0572],
         [-0.2495,  0.1740, -0.3572],
         [ 0.3496,  0.3546,  0.2610],
         [ 0.2826,  0.4353, -0.3105],
         [-0.1331, -0.0448, -0.2625],
         [ 0.3561,  0.1659, -0.1571],
         [-0.1036, -0.0262, -0.2957],
         [ 0.0132,  0.1542, -0.4864],
         [ 0.2793,  0.2334, -0.1412],
         [-0.1528, -0.0184, -0.2182],
         [-0.1823, -0.0512, -0.2283],
         [-0.1329, -0.0350, -0.2510]]], grad_fn=<AddBackward0>) torch.Size([1, 20, 3])
Batch 1 complete. Time taken: load(0.0131), train(18.6), total(18.6). 


Exception: 

In [62]:
def active_logits(raw_logits, word_ids):
    word_ids = word_ids.view(-1)
    active_mask = word_ids.unsqueeze(1).expand(word_ids.shape[0], 3)
    print(active_mask)
    active_mask = active_mask != -1
    active_logits = raw_logits.view(-1, 3)
    print(active_logits)
    active_logits = torch.masked_select(active_logits, active_mask) # return 1dTensor
    active_logits = active_logits.view(-1, 3) 
    return active_logits

In [66]:
word_ids = tokenizer('Hi my name is سیس Yousef').word_ids()
L = len(word_ids)
word_ids = [word_id if word_id is not None else -1 for word_id in word_ids]
word_ids = torch.tensor(word_ids)

raw_logits = torch.rand((L, 3))
active_logits(raw_logits, word_ids)

tensor([[-1, -1, -1],
        [ 0,  0,  0],
        [ 1,  1,  1],
        [ 2,  2,  2],
        [ 3,  3,  3],
        [ 4,  4,  4],
        [ 4,  4,  4],
        [ 5,  5,  5],
        [ 5,  5,  5],
        [ 5,  5,  5],
        [-1, -1, -1]])
tensor([[0.9573, 0.3607, 0.6314],
        [0.4865, 0.2565, 0.6984],
        [0.4938, 0.6211, 0.6231],
        [0.1811, 0.8638, 0.6316],
        [0.6443, 0.3207, 0.4406],
        [0.1913, 0.6350, 0.8052],
        [0.2440, 0.1049, 0.7696],
        [0.5300, 0.6490, 0.8611],
        [0.2157, 0.6968, 0.4399],
        [0.8934, 0.0529, 0.4582],
        [0.0897, 0.0093, 0.6105]])


tensor([[0.4865, 0.2565, 0.6984],
        [0.4938, 0.6211, 0.6231],
        [0.1811, 0.8638, 0.6316],
        [0.6443, 0.3207, 0.4406],
        [0.1913, 0.6350, 0.8052],
        [0.2440, 0.1049, 0.7696],
        [0.5300, 0.6490, 0.8611],
        [0.2157, 0.6968, 0.4399],
        [0.8934, 0.0529, 0.4582]])

In [80]:
from torch.utils.data import Dataset
# need help with this
IGNORE_INDEX = -100
NON_LABEL = -1
LABELS_TO_IDS = {
    'O': 0,
    'B-PERS': 1,
    'I-PERS': 2
}

df = pd.DataFrame({
        'text': texts,
        'entities': labels
    })

SUBTOKENS = True
class FeedbackPrizeDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, has_labels):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.has_labels = has_labels
    
    def __getitem__(self, index):
        text = self.data.text[index]
        encoding = self.tokenizer(
            text.split(),
            is_split_into_words = True,
            padding = 'max_length',
            truncation = True,
            max_length = self.max_len
        )
        word_ids = encoding.word_ids()

        # targets
        if self.has_labels:
            word_labels = self.data.entities[index]
            prev_word_idx = None
            labels_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    labels_ids.append(IGNORE_INDEX)
                elif word_idx != prev_word_idx:
                    labels_ids.append(LABELS_TO_IDS[word_labels[word_idx]])
                else:
                    if SUBTOKENS:
                        labels_ids.append(LABELS_TO_IDS[word_labels[word_idx]])
                    else:
                        labels_ids.append(IGNORE_INDEX)
                prev_word_idx = word_idx
            encoding['labels'] = labels_ids
        # convert to torch.tensor
        item = {k: torch.as_tensor(v) for k, v in encoding.items()}
        word_ids2 = [w if w is not None else NON_LABEL for w in word_ids]
        item['word_ids'] = torch.as_tensor(word_ids2)
        return item

    def __len__(self):
        return self.len
    
dset = FeedbackPrizeDataset(df.rename(columns={'labels':'entities'}),  tokenizer,  10,True)

In [82]:
def active_labels(labels):
    active_mask = labels.view(-1) != IGNORE_INDEX
    active_labels = torch.masked_select(labels.view(-1), active_mask)
    return active_labels

In [83]:
for item in dset:
    input_ids, attention_mask, labels, word_ids = item.values()
    print(labels)
    print(active_labels(labels))
    break

tensor([-100,    0,    0,    0,    0,    1,    1,    1,    2, -100])
tensor([0, 0, 0, 0, 1, 1, 1, 2])
